# Twitter Data Sentiment Analysis

Twitter tweets are an attractive data source to analyze
as Twitter users tweet about many different topics in which
they not only impart knowledge to others but also express
their feelings and opinions. Analyzing this data can result
in valuable insights and can be useful to detect trends and
drive business decisions. Notebooks are a powerful platform
for data scientists to analyze Twitter data.

## Auto Industry Tweets

This notebook analyzes Twitter data to glean insights about
the automotive industry. As the automotive industry is one of
the largest industries in the world and still very much a growth
industry, analyzing tweets about cars can assist manufacturers
to pay closer attention to market dynamics and position their
companies to take advantage of demographic changes and shifts
in consumer expectations.

## Learning goals

The notebook is structured into different sections. In the first
sections, you will perform a general analysis on the data set then
you will go deeper in the analysis to gain meaningful insights
about manufacturers.

1. Determine the countries with the highest number of tweets
   (based on the user profile information).

2. Analyze tweet sentiments

3. Draw insights from tweets about major car manufacturers worldwide
   by combining Twitter timeline analysis with sentiment, gender
   distribution and location distribution.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not use this file except in compliance with the License. You may obtain
# a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.

In [ ]:
# @hidden_cell
#
# The hidden_cell magic allows us to avoid exposing credentials in shared
# output, but you still should be careful how you save, share or allow access
# to the notebook.

# SET-UP: Add credentials, URL and prefix for dashDB here.

DASHDB_USERNAME = "<username>"
DASHDB_PASSWORD = "<password>"
DASHDB_JDBC_URL = "<ssl-jdbc-url>"
DASHDB_PREFIX = "dsx_twitter"
DASHDB_SCHEMA = DASHDB_USERNAME.upper()

In [ ]:
%%capture captured_io

# Captured_io magic is used here so that the output of the pip install
# does not have to show in the final report. To check the output, run
# the next cell with DEBUG=False. This install may require restarting
# the kernel and if you restart the kernel, then you should start
# at the top of the notebook again.

!pip install wordcloud

In [ ]:
# You can use this cell to see the captured IO from the above cell.
# After seeing that the captured_io from above is OK, then you can set
# DEBUG = False (and re-run this cell) to make the final report look cleaner.
# DEBUG will also be used below to hide some output that you may want while
# getting started, but may not want in the final exported results.
DEBUG = False
if DEBUG:
    captured_io()  # noqa F821

In [ ]:
# Import and initialize modules

from pyspark.sql import SQLContext
from pyspark.sql import functions as F

from operator import add
import re
import time
from datetime import date
from dateutil import parser

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
from IPython.core.display import Javascript

from __future__ import division

%matplotlib inline

sqlContext = SQLContext(sc)  # noqa

In [ ]:
# Defining global variables and helper functions
#
# car_makers_list: An array of car manufacturers. Each element in the array is
#                  a list with all spelling variants.
# car_makers_name_list: List of the preferred uppercase names to use for car
#                       makers.
# num_car_makers: The number of car makers expected.

car_makers_list = [['bmw'],
                   ['daimler', 'mercedes'],
                   ['gm', 'general motors'],
                   ['tesla'],
                   ['toyota'],
                   ['vw', 'volkswagen']]
num_car_makers = len(car_makers_list)

car_makers_name_list = []
for car_maker in car_makers_list:
    car_makers_name_list.append(car_maker[0].upper())

# Plotting variables
ind = np.arange(num_car_makers)  # index list for plotting
width = 0.8  # the width of the bars in the bar plots

In [ ]:
# Helper functions

# The helper function called GeoChart is used to plot the world map in a DOM
# element (an iframe).
def GeoChart(data_string, element):
    return Javascript("""
        //container.show();
        function draw() {{
          var chart = new google.visualization.GeoChart(document.getElementById(""" + element + """));
          chart.draw(google.visualization.arrayToDataTable(""" + data_string + """));
        }}
        google.load('visualization', '1.0',
                    {'callback': draw, 'packages':['geochart']});
        """, lib="https://www.google.com/jsapi")


# The helper function addMissingDates checks for any missing dates in
# DataFrames with time series data.
#   * baseDataframe: This DataFrame contains all dates. It must have the
#                    column names [POSTING_TIME, NUM_TWEETS]
#   * checkedDataframe: This DataFrame contains the dates that need to be
#                       checked. It must have the column names
#                       [POSTING_TIME, NUM_TWEETS]
def addMissingDates(baseDates, checkedDates):
    temp = checkedDates.copy()
    checkedDatesValues = checkedDates['POSTING_TIME']
    for index, row in baseDates.iterrows():
        if not row['POSTING_TIME'] in checkedDatesValues.tolist():
            row['NUM_TWEETS'] = 0
            temp = temp.append(row)
    return temp.sort_values('POSTING_TIME')

## The data set

In [ ]:
# Loading data into Spark DataFrames
#
# To access the data in dashDB, you must provide the dashDB service
# credentials. These were set in the first input cell to make them obvious
# and easy to find. The data is retrieved from the database by using the
# Spark JDBC connector and is loaded into a Spark DataFrame in the notebook
# called df_CARS_TWEETS using sqlContext.read.jdbc. The DataFrame has the same
# column names as the tweets table in dashDB.

props = {}
props['user'] = DASHDB_USERNAME
props['password'] = DASHDB_PASSWORD
jdbcurl = DASHDB_JDBC_URL

# Get the data frame
df_TWEETS = sqlContext.read.jdbc(jdbcurl, DASHDB_SCHEMA + '.' +
                                 DASHDB_PREFIX + '_' + 'TWEETS',
                                 properties=props)
df_SENTIMENTS = sqlContext.read.jdbc(jdbcurl, DASHDB_SCHEMA + '.' +
                                     DASHDB_PREFIX + '_' + 'SENTIMENTS',
                                     properties=props)

if DEBUG:
    df_TWEETS.printSchema()
    df_SENTIMENTS.printSchema()

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType, FloatType, StringType, StructField, StructType, TimestampType  # noqa
udf = UserDefinedFunction(
    lambda x: 0 if x == 'AMBIVALENT' else 1 if x == 'POSITIVE' else -1,
    IntegerType())
udf2 = UserDefinedFunction(
    lambda x: 'POSITIVE' if x > 0 else 'NEGATIVE' if x < 0 else 'AMBIVALENT',
    StringType())

df = df_TWEETS.join(df_SENTIMENTS, "MESSAGE_ID")
df = df.withColumn('SENTIMENT_POLARITY', udf(df.SENTIMENT_POLARITY)).groupBy(
    'MESSAGE_ID').agg(F.mean('SENTIMENT_POLARITY').alias("SENTIMENT_POLARITY"))
df = df.withColumn('SENTIMENT', udf2(df.SENTIMENT_POLARITY))
df_JOIN_TWEETS = df_TWEETS.join(df, "MESSAGE_ID")

if DEBUG:
    df_JOIN_TWEETS.printSchema()
    
# The following code cell counts the number of rows which were loaded into the
# DataFrame which is equivalent to the number of tweets available for further
# processing.
print("Number of tweets: " + str(df_TWEETS.count()))
print("Number of sentiment records: " + str(df_SENTIMENTS.count()))    
print(
    "Aggregated tweets with sentiment scores: " + str(df_JOIN_TWEETS.count()))

In [ ]:
# Transforming the data
#
# You can't analyze the data that you have just loaded into the data frames
# the way it is. You must first transform the data. The output of the data
# transformation process is a new Spark DataFrame which has the target
# structure on which to base the data analysis. This Spark DataFrame called
# df_cleaned_tweets functions as the main data source for all further
# processing.
#
# Run the next cell to carry out the following transformations on the data:
# 
# 1) Remove the time from the timestamp values as only the date information is
#    relevant.
#
# 2) Change the values in the string columns like user country, state and city
#    to upper case.
#
# 3) Change the tweet posting location information from a string
#    ('pos (42.000 42.000)') to a numeric value represented by the longitude
#     and latitude coordinates. 
#
# You will use the resulting DataFrame (df_cleaned_tweets) as the base data
# source for all further step. The sample uses Spark to do all heavy
# computation. When it is time to plot or collect the results, the returned
# data is copied into the kernel memory, in other words moved from a Spark
# DataFrame to a pandas DataFrame.


def getLongitudeLatitude(position):
    parts = str(position).split('(')[1].split(')')[0].split(' ')
    return parts


def getLongitude(row):
    if row.MESSAGE_LOCATION is None:
        return None
    else:
        try:
            parts = getLongitudeLatitude(row.MESSAGE_LOCATION)
            lon = float(parts[0])
            return lon
        except:
            return None


def getLatitude(row):
    if row.MESSAGE_LOCATION is None:
        return None
    else:
        try:
            parts = getLongitudeLatitude(row.MESSAGE_LOCATION)
            lon = float(parts[1])
            return lon
        except Exception:
            return None


def getDateIgnoreTime(row):
    posting_time = parser.parse(str(row.MESSAGE_POSTED_TIME))
    posting_time = posting_time.replace(
        hour=0, minute=0, second=0, microsecond=0)
    return posting_time


schema = StructType([StructField('MESSAGE_ID', StringType()),
                     StructField('POSTING_TIME', TimestampType()),
                     StructField('MESSAGE_BODY', StringType()),
                     StructField('USER_GENDER', StringType()),
                     StructField('USER_STATE', StringType()),
                     StructField('USER_COUNTRY', StringType()),
                     StructField('USER_CITY', StringType()),
                     StructField('MESSAGE_LANGUAGE', StringType()),
                     StructField('MESSAGE_LOCATION_LONGITUDE', FloatType()),
                     StructField('MESSAGE_LOCATION_LATITUDE', FloatType()),
                     StructField('SENTIMENT', StringType()),
                     StructField('USER_FOLLOWERS_COUNT', IntegerType()),
                     StructField('USER_FRIENDS_COUNT', IntegerType())])

df_cleaned_tweets = sqlContext.createDataFrame(df_JOIN_TWEETS.map(lambda row: [
    row.MESSAGE_ID,
    getDateIgnoreTime(row),
    row.MESSAGE_BODY,
    row.USER_GENDER,
    unicode(row.USER_STATE).upper(),
    unicode(row.USER_COUNTRY).upper(),
    unicode(row.USER_CITY).upper(),
    row.MESSAGE_LANGUAGE,
    getLongitude(row),
    getLatitude(row),
    row.SENTIMENT,
    row.USER_FOLLOWERS_COUNT,
    row.USER_FRIENDS_COUNT
]), schema)

df_cleaned_tweets.registerTempTable(DASHDB_PREFIX + '_TWEETS_CLEANED')
if DEBUG:
    df_cleaned_tweets.printSchema()
df_cleaned_tweets.cache();  # End in semi-colon to suppress output.

## Distribution of tweets by country

In [ ]:
# This section shows you how to extract the countries, which have the highest
# number of tweets. To do that the data is grouped according to the
# USER_COUNTRY column and the rows in each group are counted. Then the
# groups are sorted in descending order.

# Group by country
df_cleaned_tweets_countries = df_cleaned_tweets.groupBy(
    'USER_COUNTRY').agg(
    F.count('MESSAGE_BODY').alias('NUM_TWEETS')).orderBy(
    'NUM_TWEETS', ascending=False)

df_cleaned_tweets_countries.cache();  # End in semi-colon to suppress output

# Show tweet count for 5 top countries
if DEBUG:
    df_cleaned_tweets_countries.show(5)

In [ ]:
p_df_cleaned_tweets_countries = df_cleaned_tweets_countries.toPandas()
p_df_cleaned_tweets_countries.ix[
    p_df_cleaned_tweets_countries['USER_COUNTRY'] == 'NONE', 'USER_COUNTRY'
] = 'UNKNOWN'

In [ ]:
# The next cells show you two ways of plotting the countries with the largest
# number of tweets. As this data needs no further transformation, the data is
# copied into a Pandas DataFrame which is used for plotting the results.

num_plotted_countries = 10

countries = p_df_cleaned_tweets_countries[
    'USER_COUNTRY'][:num_plotted_countries]
num_tweets = p_df_cleaned_tweets_countries[
    'NUM_TWEETS'][:num_plotted_countries]
y_pos = np.arange(len(countries))
colors = np.repeat('b', num_plotted_countries - 1).tolist()
colors = ['gray'] + colors

plt.figure(figsize=(10, 8))
plt.barh(y_pos, num_tweets, align='center', color=colors)
plt.yticks(y_pos, countries)
plt.xlabel('Number of Tweets')
plt.title('Tweets Country Distribution based on the User Profile')
plt.ylim(-1, len(y_pos))

plt.show()

%%html
<div id="plot_div" style="width: 900px; height: 500px;"></div>

In [ ]:
countries = p_df_cleaned_tweets_countries['USER_COUNTRY']
num_tweets = p_df_cleaned_tweets_countries['NUM_TWEETS']

data = "[['Country', 'Num Tweets']"
index = 0
for country in countries:
    country = country.replace("'", "")
    data = data + ", ['" + country + "', " + str(num_tweets[index]) + "]"
    index += 1
data += "]"

GeoChart(data, "'plot_div'")

In [ ]:
# As you won't need the data frames related to countries any longer run the
# following cell to clear the memory of those variables (both Spark and
# pandas).

df_cleaned_tweets_countries.unpersist()
df_cleaned_tweets_countries = None
p_df_cleaned_tweets_countries = None

## Analyzing tweet sentiments

Sentiment analysis is one of the most valuable sources of information
that the IBM twitter API provides. By giving each tweet a sentiment
value, you can determine whether the content of a tweet is positive,
negative, ambivalent, neutral, or NULL, if no value is provided by
the API. Unfortunately, a sentiment value is provided for English,
German, French, and Spanish tweets only. As the data set also has tweets
in other languages, only a subset of the tweets in the data set have
a sentiment value.

In [ ]:
# Get number of tweets with P N U sentiment by grouping the sentiment value.
tweets_grouped_by_sentiment = df_cleaned_tweets.groupBy(
    'SENTIMENT').agg(F.count('MESSAGE_ID').alias('NUM_TWEETS'))

tweets_grouped_by_sentiment.cache()

# Show the tweet counts by sentiment (text output)
if DEBUG:
    tweets_grouped_by_sentiment.show(5)

# Move the results to pandas
p_tweets_grouped_by_sentiment = tweets_grouped_by_sentiment.toPandas()

In [ ]:
# This cell plots the sentiment values of all tweets in the data set.

# Data plot 1
plot1_labels = p_tweets_grouped_by_sentiment['SENTIMENT']
plot1_values = p_tweets_grouped_by_sentiment['NUM_TWEETS']
plot1_colors = ['blue', 'red', 'gray', 'yellow', 'green']

# Data plot 2
cond1 = (p_tweets_grouped_by_sentiment['SENTIMENT'] == 'POSITIVE')
cond2 = (p_tweets_grouped_by_sentiment['SENTIMENT'] == 'NEGATIVE')

pMessage_sentiment_statistics_defined = p_tweets_grouped_by_sentiment[
    cond1 | cond2]
plot2_labels = pMessage_sentiment_statistics_defined['SENTIMENT']
plot2_values = pMessage_sentiment_statistics_defined['NUM_TWEETS']
plot2_colors = ['blue', 'red']

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(23, 10))

axes[0].pie(
    plot1_values, labels=plot1_labels, colors=plot1_colors, autopct='%1.1f%%')
axes[0].set_title('Percentage of Sentiment Values in all Tweets')
axes[0].set_aspect('equal')
axes[0].legend(loc="upper right", labels=plot1_labels)

# Plot
axes[1].pie(
    plot2_values, labels=plot2_labels, colors=plot2_colors, autopct='%1.1f%%')
axes[1].set_title(
    'Percentage of Positive and Negative Sentiment Values in all Tweets')
axes[1].set_aspect('equal')
axes[1].legend(loc="upper right", labels=plot2_labels)

fig.subplots_adjust(hspace=1)
plt.show()

In [ ]:
# After you have plotted the results, run this cell to release the memory of
# the variables you used.

tweets_grouped_by_sentiment.unpersist()
tweets_grouped_by_sentiment = None
p_tweets_grouped_by_sentiment = None

## Analyzing tweet timelines

To learn more about which car manufacturing events occurred in 2015, you can
plot data over time. The following section groups all tweets created in 2015
by their posting date (and sentiment value) and counts the number of tweets
per date.

In [ ]:
# Group by year-month-day and the sentiment
df_tweets_and_sentiment_over_time = df_cleaned_tweets.groupBy(
    'POSTING_TIME', 'SENTIMENT').agg(
    F.count('MESSAGE_BODY').alias('NUM_TWEETS')).orderBy(
    'POSTING_TIME', ascending=True)

# Group by year-month-day
df_tweets_over_time = df_tweets_and_sentiment_over_time.groupBy(
    'POSTING_TIME').agg(
    F.sum('NUM_TWEETS').alias('NUM_TWEETS')).orderBy(
    'POSTING_TIME', ascending=True)

# Move to Pandas
p_df_tweets_and_sentiment_over_time = (
    df_tweets_and_sentiment_over_time.toPandas())
p_df_tweets_over_time = df_tweets_over_time.toPandas()

In [ ]:
# Run the next cell to prepare the data for plotting by moving data with
# different sentiment values into different data frames.

p = p_df_tweets_and_sentiment_over_time

positive_mask = p['SENTIMENT'] == 'POSITIVE'
negative_mask = p['SENTIMENT'] == 'NEGATIVE'
neutral_mask = p['SENTIMENT'] == 'NEUTRAL'
ambivalent_mask = p['SENTIMENT'] == 'AMBIVALENT'
null_mask = p['SENTIMENT'].isnull()

p_df_tweets_and_sentiment_over_time_positive = p[positive_mask]
p_df_tweets_and_sentiment_over_time_negative = p[negative_mask]
p_df_tweets_and_sentiment_over_time_neutral = p[neutral_mask]
p_df_tweets_and_sentiment_over_time_ambivalent = p[ambivalent_mask]
p_df_tweets_and_sentiment_over_time_null = p[null_mask]

p_df_num_tweets_and_sentiment_over_time_positive = addMissingDates(
    p_df_tweets_over_time, p_df_tweets_and_sentiment_over_time_positive)
p_df_num_tweets_and_sentiment_over_time_negative = addMissingDates(
    p_df_tweets_over_time, p_df_tweets_and_sentiment_over_time_negative)
p_df_num_tweets_and_sentiment_over_time_neutral = addMissingDates(
    p_df_tweets_over_time, p_df_tweets_and_sentiment_over_time_neutral)
p_df_num_tweets_and_sentiment_over_time_ambivalent = addMissingDates(
    p_df_tweets_over_time, p_df_tweets_and_sentiment_over_time_ambivalent)
p_df_num_tweets_and_sentiment_over_time_null = addMissingDates(
    p_df_tweets_over_time, p_df_tweets_and_sentiment_over_time_null)

In [ ]:
# Run the next cell to plot your results after data transformation. First plot
# the number of tweets about Volkswagen, Toyota, BMW, Daimler, and General
# Motors spread across 2015, then plot their positive and negative sentiment
# values, and lastly plot only the sentiment value applied to the number of
# tweets.

# Take the beginning of each month
mask_1day = p_df_tweets_over_time['POSTING_TIME'].map(lambda x: x.day) == 1
x = p_df_tweets_over_time[mask_1day]['POSTING_TIME']
y = p_df_tweets_over_time['NUM_TWEETS']

# Positive preparation
py = p_df_num_tweets_and_sentiment_over_time_positive['NUM_TWEETS']
# Negative preparation
ny = p_df_num_tweets_and_sentiment_over_time_negative['NUM_TWEETS']
# Undefined preparation
ney = p_df_num_tweets_and_sentiment_over_time_neutral['NUM_TWEETS']
# Ambivalent preparation
ay = p_df_num_tweets_and_sentiment_over_time_ambivalent['NUM_TWEETS']
# Null preparation - undefined
nully = p_df_num_tweets_and_sentiment_over_time_null['NUM_TWEETS']

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

# Plot1
axes[0].plot(range(len(y)), y, linewidth=2)
axes[0].set_xticks(x.index.tolist())
axes[0].set_xticklabels([day.strftime("%Y-%m-%d") for day in x])
axes[0].margins = 0
axes[0].set_xlabel('Date/Time')
axes[0].set_ylabel('Num of Tweets')
axes[0].set_title('Number of Tweets Over Time - ALL TWEETS')
axes[0].set_xlim(0, len(y))
axes[0].legend(loc="upper right", labels=['All Tweets'])

axes[1].plot(range(len(y)), y, linewidth=2, color='blue')
axes[1].plot(range(len(py)), py, linewidth=2, color='green')
axes[1].plot(range(len(ny)), ny, linewidth=2, color='red')

axes[1].set_xticks(x.index.tolist())
axes[1].set_xticklabels([day.strftime("%Y-%m-%d") for day in x])
axes[1].margins = 0
axes[1].set_xlabel('Date/Time')
axes[1].set_ylabel('Num of Tweets')
axes[1].set_title('Number of Tweets Over Time - All, Positive and Negative')
axes[1].set_xlim(0, len(y))
axes[1].legend(loc="upper right",
               labels=[
                   'All Tweets', 'Positive', 'Negative', 'Undefined Sentiment'
               ])

fig.subplots_adjust(hspace=1)
plt.show()

In [ ]:
# After you've completed plotting the results, clear the memory of the
# variables you used.

df_tweets_and_sentiment_over_time.unpersist()
df_tweets_over_time.unpersist()

df_tweets_and_sentiment_over_time = None
df_tweets_over_time = None

p_df_tweets_and_sentiment_over_time = None
# We will need the variable p_df_tweets_over_time

p_df_tweets_and_sentiment_over_time_positive = None
p_df_tweets_and_sentiment_over_time_negative = None
p_df_tweets_and_sentiment_over_time_neutral = None
p_df_tweets_and_sentiment_over_time_ambivalent = None
p_df_tweets_and_sentiment_over_time_null = None

## Insights from tweets about car manufacturers

This section combines different types of analyses to dig deeper
into the list of car manufacturers (Volkswagen, Toyota, BMW,
Daimler, and  General Motors). The purpose of the analyses is to
obtain car manufacturer-based insights from tweets that could be
interesting and useful to detect potential car buyers. The first
step is to detect the tweets that mention certain car manufacturers.

In [ ]:
# Run this cell to transform the data into a new data frame that has additional
# columns for each car manufacturer. The value in this column in each row
# indicates whether the car company was mentioned in a tweet or not. 
#
# This new data frame is the new data source for subsequent computations.


def hasWord(message, word):
    return word in message


def checkCarMaker(message):
    tmp = []
    for car_maker_list_var in car_makers_list:
        contain = False
        for car_maker in car_maker_list_var:
            if hasWord(message, car_maker):
                contain = True
        tmp.extend([contain])
    return tmp


def checkCarFeatures(message, feature_list):
    tmp = []    
    contain = False
    for term in feature_list:
        if hasWord(message, term.decode('utf8')):
            contain = True
    tmp.extend([contain])
    return tmp


def getInfluence(tweet):
    return (tweet.USER_FOLLOWERS_COUNT + tweet.USER_FRIENDS_COUNT) / 2


def getAllAttributes(tweet):
    message = unicode(tweet.MESSAGE_BODY).lower()
    
    # Message id and line
    tmp = [tweet.MESSAGE_ID, tweet.MESSAGE_BODY, tweet.SENTIMENT,
           tweet.USER_GENDER, unicode(tweet.USER_COUNTRY).upper(),
           tweet.POSTING_TIME, getInfluence(tweet)]

    # Competitors in line
    tmp.extend(checkCarMaker(message)) 
    return tmp


columns_names = ['MESSAGE_ID', 'MESSAGE_BODY', 'SENTIMENT', 'USER_GENDER',
                 'USER_COUNTRY', 'POSTING_TIME', 'INFLUENCE']
for carMakerName in car_makers_name_list:
        columns_names.append(carMakerName)

df_tweets_car_maker = sqlContext.createDataFrame(df_cleaned_tweets.map(
    lambda x: getAllAttributes(x)), columns_names)

df_tweets_car_maker.cache();

if DEBUG:
    df_tweets_car_maker.printSchema()

## Analyzing tweet timelines by car maker

In [ ]:
# You will plot the data over time according to each car manufacturer. This
# can be done by using the Spark data frame that you just constructed. You
# will filter the data by each car maker and then regroup the resulting
# data frame according to the posting time of the tweets.

if DEBUG:
    print('Calculating the number of tweets over 2015 for each car maker:')

car_maker_results_list = []
for car_maker in car_makers_name_list:
    # Get car maker dataframe
    df_car_maker = df_tweets_car_maker.filter(
        df_tweets_car_maker[car_maker] == True)
    overall_car_maker_time_data = df_car_maker.groupBy(
        'POSTING_TIME').agg(
        F.count('MESSAGE_ID').alias('COUNT')).orderBy(
        'POSTING_TIME', ascending=True)

    p_overall_car_maker_time_data = overall_car_maker_time_data.toPandas()
    car_maker_results_list.append(p_overall_car_maker_time_data)
    
    overall_car_maker_time_data.unpersist()
    
    # Print to show progress while cell runs
    if DEBUG:
        print('Done for ' + car_maker)

In [ ]:
mask_1day = p_df_tweets_over_time['POSTING_TIME'].map(lambda x: x.day) == 1
x = p_df_tweets_over_time[mask_1day]['POSTING_TIME']

# Plotting
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
colors = ['blue', 'red', 'green', 'yellow', 'pink', 'black']

for i in range(0, len(car_makers_name_list)):
    y = car_maker_results_list[i]['COUNT']
    axes.plot(range(len(y)), y, linewidth=2, color=colors[i])

axes.set_xticks(x.index.tolist())
axes.set_xticklabels([day.strftime("%Y-%m-%d") for day in x])
axes.margins = 0
axes.set_xlabel('Date/Time')
axes.set_ylabel('Num of Tweets')
axes.set_title(
    'Number of Tweets Over Time according to a Car Manufacturer- ALL TWEETS')
axes.set_xlim(0, len(car_maker_results_list[0]))
axes.legend(loc="upper right", labels=car_makers_name_list)

fig.subplots_adjust(hspace=1)
plt.show()

## Explaining the peak of tweets for VW
### September 15, 2015 - October 15, 2015

In [ ]:
df_tweets_debacle = df_tweets_car_maker.filter(
    (df_tweets_car_maker["VW"] == True) & (
        df_tweets_car_maker.POSTING_TIME > '2015-09-15 0.0.0') & (
        df_tweets_car_maker.POSTING_TIME < '2015-10-15 0.0.0'))
if DEBUG:
    df_tweets_debacle.count()

In [ ]:
stop_words = set(STOPWORDS)
stop_words.add("says")
stop_words.add("over")
stop_words.add("will")

tagsRDD = df_tweets_debacle.flatMap(
    lambda t: re.split("\s", t.MESSAGE_BODY)).filter(
    lambda word: not word.startswith("http") and 
    all(ord(c) < 128 for c in word) and 
    word not in stop_words and
    len(word) > 3).map(
    lambda word: (word, 1)).reduceByKey(add, 10).map(
    lambda (a, b): (b, a)).sortByKey(False).map(
    lambda (a, b): (b, a))

# Get top words for word cloud (stopwords, etc. were removed above)
toptags = tagsRDD.take(20)

# Take 'Volkswagen' out of the pie chart keywords and then keep top ten.
top10tags = [t for t in toptags if 'Volkswagen' not in t[0]]

params = plt.gcf()
plSize = params.get_size_inches()
params.set_size_inches((plSize[0] * 2, plSize[1] * 2))

labels = [i[0] for i in top10tags]
sizes = [int(i[1]) for i in top10tags]
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral', "beige",
          "paleturquoise", "pink", "lightyellow", "coral"]

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True,
        startangle=90)

plt.axis('equal')
plt.show()

In [ ]:
# Generate a word cloud image
word_cloud = WordCloud(background_color='white',
                       normalize_plurals=True,
                       collocations=True).fit_words(dict(toptags))

plt.figure(figsize=(12, 10))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
car_maker_results_list = []

## Sentiment analysis by car maker

In [ ]:
if DEBUG:
    print('Calculating the number of positive and negative tweets over 2015 '
          'for each car maker:')

car_maker_results_list = []
for car_maker in car_makers_name_list:

    df_car_maker = df_tweets_car_maker.filter(
        df_tweets_car_maker[car_maker] == True)
    
    time_sentiment_car_maker_data = df_car_maker.groupBy(
        'POSTING_TIME', 'SENTIMENT').agg(
        F.count('MESSAGE_ID').alias('COUNT')).orderBy(
        'POSTING_TIME', ascending=True)
    
    time_sentiment_car_maker_data.cache()
    
    time_pos_sentiment_car_maker_data = time_sentiment_car_maker_data.filter(
        time_sentiment_car_maker_data['SENTIMENT'] == 'POSITIVE')
    time_neg_sentiment_car_maker_data = time_sentiment_car_maker_data.filter(
        time_sentiment_car_maker_data['SENTIMENT'] == 'NEGATIVE')

    p_time_pos_sentiment_maker = time_pos_sentiment_car_maker_data.toPandas()
    p_time_neg_sentiment_maker = time_neg_sentiment_car_maker_data.toPandas()
    
    # Collect results
    car_maker_results_list.append([p_time_pos_sentiment_maker,
                                   p_time_neg_sentiment_maker])
    time_sentiment_car_maker_data.unpersist()
    
    # Print to show progress
    if DEBUG:
        print('Done for ' + car_maker)

In [ ]:
mask_1day = p_df_tweets_over_time['POSTING_TIME'].map(lambda x: x.day) == 1
x = p_df_tweets_over_time[mask_1day]['POSTING_TIME']

for i in range(0, len(car_makers_name_list)):
    car_maker_results_list[i][0] = addMissingDates(
        p_df_tweets_over_time, car_maker_results_list[i][0])
    car_maker_results_list[i][1] = addMissingDates(
        p_df_tweets_over_time, car_maker_results_list[i][1])

fig, axes = plt.subplots(
    nrows=len(car_maker_results_list), ncols=1, figsize=(20, 15))
colors = ['blue', 'red', 'green', 'yellow', 'black']


for i in range(0, len(car_makers_name_list)):
    y1 = car_maker_results_list[i][0]['COUNT']
    y2 = car_maker_results_list[i][1]['COUNT']
    
    axes[i].plot(range(len(y1)), y1, linewidth=2, color='green')
    axes[i].plot(range(len(y2)), y2, linewidth=1, color='red')
    axes[i].set_xticks(x.index.tolist())
    axes[i].set_xticklabels([day.strftime("%Y-%m-%d") for day in x])
    axes[i].margins = 0
    axes[i].set_xlabel('Date/Time')
    axes[i].set_ylabel('Num of Tweets')
    axes[i].set_title('Number of Tweets about ' + car_makers_name_list[i] + 
                      ' Over Time - Positive and Negative')
    axes[i].set_xlim(0, len(car_maker_results_list[0][0]))
    axes[i].legend(loc="upper right", labels=['Positive', 'Negative'])

fig.subplots_adjust(hspace=1)
plt.show()

## Tweet distribution by car manufacturer

In [ ]:
# This code cell calculates the number of tweets about certain car
# manufacturers

if DEBUG:
    print('Calculating the number of tweets that mention one of the car '
          'makers:')

car_maker_tweets_count = []
for car_maker in car_makers_name_list:
    df_car_maker = df_tweets_car_maker.filter(
        df_tweets_car_maker[car_maker] == True)
    car_maker_tweets_count.append(df_car_maker.count())
    # Print to show progress
    if DEBUG:
        print('Done for ' + car_maker)

In [ ]:
# Plot the resulting numbers in a bar chart and as percentages in a pie chart. 

# Plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(23, 8))

axes[0].bar(ind, car_maker_tweets_count, width, color='b', align='center')
axes[0].set_ylabel('Num Tweets')
axes[0].set_title('Number of tweets that mention a certain Car Manufacturer')
axes[0].set_xticks(ind)
axes[0].set_xticklabels(car_makers_name_list)

# Plot
axes[1].pie(car_maker_tweets_count,
            autopct='%1.1f%%',
            labels=car_makers_name_list)
axes[1].set_aspect('equal')
axes[1].set_title('Percentage of tweets that mention a certain Car '
                  'Manufacturer')

fig.subplots_adjust(hspace=1)
plt.show()

## Tweet distribution by sentiment by car maker

In [ ]:
# Calculate the number of positive and negative tweets by car maker

positive_sum_car_makers = []
negative_sum_car_makers = []

for i in range(0, len(car_makers_name_list)):
    car_maker = car_makers_name_list[i]
    car_maker_results_list[i][0] = car_maker_results_list[i][0].fillna(0)
    car_maker_results_list[i][1] = car_maker_results_list[i][1].fillna(0)
    positive_sum_car_makers.extend(
        [car_maker_results_list[i][0]['COUNT'].sum()])
    negative_sum_car_makers.extend(
        [car_maker_results_list[i][1]['COUNT'].sum()])

In [ ]:
# Plot the results next to each other.

sum_value_P_N = np.add(positive_sum_car_makers, negative_sum_car_makers)
competitors_list_rest = np.subtract(car_maker_tweets_count, sum_value_P_N)

# Plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(23, 8))

axes[0].bar(ind, positive_sum_car_makers, width, color='b', align='center')
axes[0].bar(ind, negative_sum_car_makers, width, color='r',
            bottom=positive_sum_car_makers, align='center')
axes[0].bar(ind, competitors_list_rest, width, color='gray', align='center',
            bottom=sum_value_P_N)
axes[0].set_ylabel('Num Tweets')
axes[0].set_title('Number of tweets that mention a certain Car Manufacturer')
axes[0].set_xticks(ind)
axes[0].set_xticklabels(car_makers_name_list)
axes[0].legend(loc="upper left", labels=['POSITIVE', 'NEGATIVE', 'Others'])

axes[1].bar(ind, positive_sum_car_makers, width, color='b', align='center')
axes[1].bar(ind, negative_sum_car_makers, width, color='r',
            bottom=positive_sum_car_makers, align='center')
axes[1].set_ylabel('Num Tweets')
axes[1].set_title('Positive / Negative tweets - Car Manufacturer')
axes[1].set_xticks(ind)
axes[1].set_xticklabels(car_makers_name_list)
axes[1].legend(loc="upper left", labels=['POSITIVE', 'NEGATIVE'])

fig.subplots_adjust(hspace=1)
plt.show()

## Gender distribution 

Another interesting insight when analyzing tweets about certain
car manufacturers and what car manufacturers might want to pay
more attention to for marketing purposes is the distribution of
tweets between male and female users.

In [ ]:
# Calculate the gender distribution for each car maker.

if DEBUG:
    print('Calculating the distribution of male and female in tweets over'
          ' 2015 for each competitor:')

car_maker_info_list_M_F = []

for car_maker in car_makers_name_list:
    df_car_maker = df_tweets_car_maker.filter(
        df_tweets_car_maker[car_maker] == True)
    car_maker_gender_data = df_car_maker.groupBy('USER_GENDER').agg(
        F.count('MESSAGE_ID').alias('COUNT'))
    car_maker_gender_data.cache()

    p_car_maker_gender_data_male = car_maker_gender_data.filter(
        car_maker_gender_data['USER_GENDER'] == 'male').toPandas()
    p_car_maker_gender_data_female = car_maker_gender_data.filter(
        car_maker_gender_data['USER_GENDER'] == 'female').toPandas()
    car_maker_info_list_M_F.append([p_car_maker_gender_data_male,
                                    p_car_maker_gender_data_female])
    car_maker_gender_data.unpersist()
    # Print to show status
    if DEBUG:
        print('Done for ' + car_maker)

In [ ]:
# Prepare data for plotting
car_maker_list_female = []
car_maker_list_male = []
for i in range(0, len(car_maker_info_list_M_F)):
    car_maker_list_female.append(0 if car_maker_info_list_M_F[i][1][
        'COUNT'].empty else car_maker_info_list_M_F[i][1]['COUNT'][0])
    car_maker_list_male.append(0 if car_maker_info_list_M_F[i][0][
        'COUNT'].empty else car_maker_info_list_M_F[i][0]['COUNT'][0])

sum_value_M_F = np.add(car_maker_list_male, car_maker_list_female)
car_maker_list_M_F_rest = np.subtract(car_maker_tweets_count, sum_value_M_F)

# Plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(23, 8))

axes[0].bar(ind, car_maker_list_male, width, color='g', align='center')
axes[0].bar(ind, car_maker_list_female, width, color='b',
            bottom=car_maker_list_male, align='center')
axes[0].bar(ind, car_maker_list_M_F_rest, width, color='gray', align='center',
            bottom=sum_value_M_F)
axes[0].set_ylabel('Num Tweets')
axes[0].set_title('Number of tweets that mention a certain Competitor')
axes[0].set_xticks(ind)
axes[0].set_xticklabels(car_makers_name_list)
axes[0].legend(loc="upper left", labels=['MALE', 'FEMALE', 'UNKNOWN'])

axes[1].bar(ind, car_maker_list_male, width, color='g', align='center')
axes[1].bar(ind, car_maker_list_female, width, color='b',
            bottom=car_maker_list_male, align='center')
axes[1].set_ylabel('Num Tweets')
axes[1].set_title('Male / Female Distribution - Competitors')
axes[1].set_xticks(ind)
axes[1].set_xticklabels(car_makers_name_list)
axes[1].legend(loc="upper left", labels=['MALE', 'FEMALE'])

fig.subplots_adjust(hspace=1)
plt.show()

## Measuring influence

In this section, we will calculate the average value of the influence
variable of the people who tweeted about a certain car maker. The
influence varable is calculated by:

$$Influence = ( \space num \space of \space followers +
\space number \space of \space friends \space ) \div 2$$

The influence score gives an indication whether someone is a famous
person or a public figure in society or whether the twitter account
is owned by the media or a company. This average value gives an
indication about the people who are interested in a certain car maker.

In [ ]:
# Returns a list of pandas DFs.
def getInsights_Influence(sparkDF, car_maker_list):
    if DEBUG:
        print('Calculating the influence insight of the users in tweets over '
              '2015 for each competitors:')

    car_maker_result_list = []
    for car_maker in car_maker_list:
        df_car_maker = sparkDF.filter(sparkDF[car_maker] == True)
        car_maker_insight_data = df_car_maker.select(F.avg('INFLUENCE').alias(
            'AVE_INFLUENCE'))
        car_maker_result_list.append(car_maker_insight_data.toPandas())
        df_car_maker.unpersist()
        car_maker_insight_data.unpersist()
        # Print to show status
        if DEBUG:
            print('Done for ' + car_maker)
    return car_maker_result_list

In [ ]:
influenceInsights = getInsights_Influence(df_tweets_car_maker,
                                          car_makers_name_list)

In [ ]:
influence_list = []
for i in range(0, len(car_makers_name_list)):
    influence_list.append(influenceInsights[i]['AVE_INFLUENCE'][0])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))

rects = axes.bar(ind, influence_list, width, color='b', align='center')
axes.set_ylabel('Average Value of the Influence Score')

# influence=(number of friends + number of followers)/2
axes.set_title('The Average of the User Influence for Each Car Maker')
axes.set_xticks(ind)
axes.set_xticklabels(car_makers_name_list)

fig.subplots_adjust(hspace=1)
plt.show()

In [ ]:
influenceInsights = None

## Distribution of tweets by country across car manufacturer

In [ ]:
# This cell calculates in which countries the most tweets were posted about
# car manufacturers. This information can support marketing and sales when
# evaluating potential customers.

if DEBUG:
    print('Calculating the country distribution of tweets over 2015 for each '
          'car maker:')

car_maker_info_list_countries = []
i = 0

for car_maker in car_makers_name_list:
    df_car_maker = df_tweets_car_maker.filter(
        df_tweets_car_maker[car_maker] == True)
    car_maker_country_data = df_car_maker.groupBy(
        'USER_COUNTRY').agg(
        F.count('MESSAGE_ID').alias('COUNT')).orderBy(
        'COUNT', ascending=False)
    car_maker_country_data.cache()

    p_car_maker_country_data = car_maker_country_data.toPandas()
    car_maker_info_list_countries.append(p_car_maker_country_data)
    car_maker_country_data.unpersist()

    # Print to show status
    if DEBUG:
        print('Done for ' + car_maker)

In [ ]:
num_taken_countries = 5

# Plot
fig, axes = plt.subplots(
    nrows=len(car_makers_name_list), ncols=1, figsize=(15, 20))


for j in range(0, len(car_makers_name_list)):
    color = 'b'  # np.random.rand(3,1)
    colors = np.repeat(color, num_taken_countries).tolist()

    country_list_num = car_maker_info_list_countries[j]['COUNT'][
        :num_taken_countries]
    country_list_labels = car_maker_info_list_countries[j]['USER_COUNTRY'][
        :num_taken_countries]

    for counter in range(0, len(country_list_labels)):
        if country_list_labels[counter] == 'NONE':
            colors[counter] = 'gray'

    axes[j].barh(np.arange(num_taken_countries),
                 country_list_num,
                 width,
                 color=colors,
                 align='center')
    axes[j].set_xlabel('Num Tweets')
    axes[j].set_title('Country Distribution for ' + car_makers_name_list[j])
    axes[j].set_yticks(ind)
    axes[j].set_yticklabels(country_list_labels.tolist())

fig.subplots_adjust(hspace=1)
plt.show()

In [ ]:
# Plot for each selected country.
us_list = []
uk_list = []
de_list = []


def getListForCountry(x, country):
    df = x[x['USER_COUNTRY'] == country]['COUNT']
    return [0] if df.empty else [df.tolist()[0]]


for i in range(0, len(car_makers_name_list)):
    x = car_maker_info_list_countries[i]
    us_list.extend(getListForCountry(x, 'UNITED STATES'))
    uk_list.extend(getListForCountry(x, 'UNITED KINGDOM'))
    de_list.extend(getListForCountry(x, 'GERMANY'))

# Plot
colors = ['r', 'g', 'b', 'w', 'pink', 'y']
us_values = us_list
uk_values = uk_list
de_values = de_list

# Plot
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(23, 10))

axes[0].pie(us_values,
            autopct='%1.1f%%', colors=colors,
            labels=car_makers_name_list)
axes[0].set_aspect('equal')
axes[0].set_title('Percentage of Tweets from Users in US among Car Makers')

# Plot
axes[1].pie(uk_values,
            autopct='%1.1f%%',
            colors=colors,
            labels=car_makers_name_list)
axes[1].set_aspect('equal')
axes[1].set_title('Percentage of Tweets from Users in UK among Car Makers')

# Plot
axes[2].pie(de_values,
            autopct='%1.1f%%', colors=colors, labels=car_makers_name_list)
axes[2].set_aspect('equal')
axes[2].set_title(
    'Percentage of Tweets from Users in GERMANY among Car Makers')

fig.subplots_adjust(hspace=1)
plt.show()

# Summary
In this notebook you learned how to use notebooks to analyze Twitter
data and extract interesting insights from tweets. You learned how to
easily perform complex computations on a large amount of data in a
notebook by using SparkContext, which enables you to start tasks on the
Spark cluster. In addition, you learned how to integrate data from dashDB
using the Spark connector and how to use Spark and pandas DataFrames.